In [ ]:
#  https://github.com/veeps/shelter_status

In [1]:
import datetime as dt
import pandas as pd
import re
import numpy as np
import nltk
import regex as reg
import matplotlib.pyplot as plt

from twitterscraper import query_tweets
from sklearn.decomposition import LatentDirichletAllocation as LDA
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import stop_words
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_multilabel_classification
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from bs4 import BeautifulSoup

pd.set_option('display.max_colwidth',-1)
pd.set_option('display.max_rows', 1500)
pd.set_option('display.max_columns', 500)

INFO: {'User-Agent': 'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16'}


## DATA CLEANING ALL LABELED TWEETS FIRE/FLOOD/HURRICANE

In [2]:
cd '/Users/Brian 1/Desktop/GA_12.10_1.04/shelter_status/data'

/Users/Brian 1/Desktop/GA_12.10_1.04/shelter_status/Data


In [3]:
df = pd.read_csv('../Data/all_tweets_clean.csv')

In [4]:
df1 = df[df.isna().any(axis=1)]

In [10]:
null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum()
print(df[df['disaster'].isnull()][null_columns])

       requesting_help disaster
10572 NaN               NaN    
13779 NaN               NaN    
18727 NaN               NaN    
28578 NaN               NaN    


In [11]:
df.iloc[10572]

text               one week after was hit by  of residents have no power have no clean water
requesting_help    NaN                                                                      
disaster           NaN                                                                      
languages          en                                                                       
Name: 10572, dtype: object

In [12]:
df.groupby('disaster')['requesting_help'].value_counts()

disaster   requesting_help
fire       0.0                468
           1.0                34 
floods     0.0                326
           1.0                153
hurricane  0.0                395
           1.0                109
Name: requesting_help, dtype: int64

In [13]:
df2 = df.dropna()

In [14]:
df3 =df2.reset_index()

In [15]:
df3=df2[['text','requesting_help','disaster']]

In [16]:
df3.reset_index(drop=True, inplace=True)

In [17]:
df3.loc[df3['requesting_help']==1]

,text,requesting_help,disaster
17,south florida about to have a very musical hurricane on our hands,1.0,hurricane
21,so will be im loaded with beer and water,1.0,hurricane
42,instead of having grey matter in brain he has straw full on brown matter from a donkey shat nuke hurricane dorian lunacy isn t a strong enough word if he only had a brain hatforbrains,1.0,hurricane
50,i was really hoping not to need my annual hurricane supplies list but looks to be heading this way stocking up today just in case,1.0,hurricane
64,a link to a gofundme for those being affected by the in the bahamas it has helped in the past with hurricane mathew please use your platform and help share it,1.0,hurricane
65,after disasters like it is inspiring to see individuals and organizations collaborating to meet the needs of those in crisis today we are thrilled to have partnered with and as we provided safe water for their work in the abacos,1.0,hurricane
68,family youve already contributed so much for us to aid those who were affected by if you would like to aid us simply lay your contribution at the altar tomorrow at am we ask that you would refrain from sending clothes however at fellowship bible baptist church,1.0,hurricane
71,we are doing our best to help the in their time of need please join us at don julio mexican grill,1.0,hurricane
78,thank you to west palm beach for donating over gallons of water for the bahamas,1.0,hurricane
80,as part of pbda s various relief efforts nd and rd grades made cards this week for the to deliver to families in the bahamas this weekend the cards will be included in care packages filled with food and hygiene products,1.0,hurricane


In [ ]:
index 17, 21, 42, 50, 107, 215, 220, 235, 237, 238, 
252, 253, 266, 274, 283, 299, 302, 303, 304,
328, 365, 368, 426, 459, 469, 471, 477, 489, 490, 514, 527, 528, 530, 532, 543, 581, 612, 633, 
636, 640, 641, 642, 645, 649, 651, 674, 696, 697, 727, 769, 784,
 yes 1 274, 636, 

In [22]:
pwd

'/Users/Brian 1/Desktop/GA_12.10_1.04/shelter_status/Data'

In [25]:
df3.to_csv('df3_brian.csv')

In [26]:
pwd

'/Users/Brian 1/Desktop/GA_12.10_1.04/shelter_status/Data'

In [3]:
df4 =pd.read_csv('df3_brian - df3_brian.csv')

In [5]:
df4['requesting_help'].value_counts()

0    1242
1    243 
Name: requesting_help, dtype: int64

In [6]:
1242+243

1485

In [8]:
1 - 243/1485

0.8363636363636364

In [9]:
df5 = df4[['text','requesting_help','disaster']]

In [16]:
df6 =df5['requesting_help']

In [ ]:
df[df['value'].astype(str).str.contains('1234.+')]

In [ ]:
#df5[df5['requesting_help'].astype(str).str.contains('need')]
df1[df1['col'].str.contains('foo', regex=False)]

In [18]:
df5[df5['text'].str.contains('need',regex=False)]

,text,requesting_help,disaster
1,after disasters like it is inspiring to see individuals and organizations collaborating to meet the needs of those in crisis today we are thrilled to have partnered with and as we provided safe water for their work in the abacos,1,hurricane
3,we are doing our best to help the in their time of need please join us at don julio mexican grill,1,hurricane
6,we re still collecting supplies to support our brothers sisters recently impacted by in the bahamas review the relief items needed and help us help them by picking up some of the items and dropping them off at the on tomorrow during worship,1,hurricane
7,forced blood drive cancellations donation center closures in impacted areas in times like these we call on donors across the u s to step up and help out to ensure blood needs can be met in storm impacted areas around the u s give,1,hurricane
10,in the wake of nick sealy thompson is embedded in the bahamas national emergency management agency s eoc working with government and relief agencies to coordinate logistic support and id critical infrastructure needs,1,hurricane
11,we need all hands on deck for our path to recovery united we will grow stronger,1,hurricane
17,headknowlesfoundation needs your help in their new location,1,hurricane
20,join fellows this sunday sept from pm at the lincoln road stage for a free benefit concert for those needing monetary help in the bahamas after being hit by,1,hurricane
23,meet the cats who survived the bahamas hurricane now theyre in and need your help,1,hurricane
24,if you haven t sent anything yet take some time this weekend and send some needed supplies to the bahamas and survivors even if it s it helps i m sending old clothes and lots of canned soups,1,hurricane


In [33]:
pipe = Pipeline([
    ('tvec', TfidfVectorizer(ngram_range=(1,2), stop_words='english')),
    ('svc', SVC(C=2,gamma='scale'))
])

params= {}

gs = GridSearchCV(pipe,params, cv=5, n_jobs=3)

In [34]:
X_train.shape

(754,)

In [35]:
y_train.shape

(754,)

In [36]:
gs.fit(X_train, y_train)
gs.score(X_train, y_train)

1.0

In [37]:
gs.score(X_test, y_test)  # C=0.9, 80.1; C=2, 82.5; 

0.9047619047619048

In [49]:
pipe2 = Pipeline([
    ('tvec', TfidfVectorizer(ngram_range=(1,2), stop_words='english')),
    ('lr', LogisticRegression(solver='liblinear'))
])


params={}

gs2 = GridSearchCV(pipe2,params, cv=5, n_jobs=3)

In [50]:
gs2.fit(X_train, y_train)
gs2.score(X_train, y_train)

0.8289124668435013

In [51]:
gs2.score(X_test,y_test)

0.8333333333333334

In [44]:
gs2.best_params_

{}

In [ ]:
# https://stackoverflow.com/questions/44066264/how-to-choose-parameters-in-tfidfvectorizer-in-sklearn-during-unsupervised-clust
If you are, for instance, using these vectors in a classification task, you can vary these parameters (and of course also the parameters of the classifier) and see which values give you the best performance.

You can do that in sklearn easily with the GridSearchCV and Pipeline objects

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stop_words)),
    ('clf', OneVsRestClassifier(MultinomialNB(
        fit_prior=True, class_prior=None))),
])
parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__estimator__alpha': (1e-2, 1e-3)
}

grid_search_tune = GridSearchCV(pipeline, parameters, cv=2, n_jobs=2, verbose=3)
grid_search_tune.fit(train_x, train_y)

print("Best parameters set:")
print grid_search_tune.best_estimator_.steps

In [45]:
params5 = {
    'tfidf_max_df':(0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1,1), (1,2), (1,3)],
    'clf__estimator__alpha':(1e-2, 1e-3)
}

In [46]:
pipe3 = Pipeline([
    ('tvec', TfidfVectorizer(ngram_range=(1,2), stop_words='english')),
    ('svc', SVC(C=2,gamma='scale'))
])



gs = GridSearchCV(pipe3,params5, cv=5, n_jobs=3)

In [47]:
gnb = GaussianNB()

In [ ]:
# Random forest

## SUPER CLEAN

In [19]:
df8 =pd.read_csv('../Data/df8_SUPER.csv')

In [20]:
df9=df8[['text','requesting_help','disaster']]

In [21]:
X = df9['text']
y= df9['requesting_help']

In [22]:
df9.groupby('disaster')['requesting_help'].value_counts()

disaster   requesting_help
fire       0                  427
           1                  75 
hurricane  0                  351
           1                  153
Name: requesting_help, dtype: int64

In [24]:
df9['requesting_help'].value_counts(normalize=True)

0    0.77336
1    0.22664
Name: requesting_help, dtype: float64

In [41]:
427+75+351+153

1006

In [42]:
75+153

228

In [44]:
# Baseline
1 - 228/1006

0.7733598409542743

## TRAIN TEST SPLIT

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y, random_state=42)

## LOGISTIC REGRESSION WITH COUNT VECTORIZER (SEE LAB 5.02)

In [26]:
# Instantiate Pipline for Logistic Regression using Count Vectorizer
# Found this terrific website that explains the different solver options in LogisticRegression
# https://towardsdatascience.com/dont-sweat-the-solver-stuff-aea7cddc3451
pipe_a = Pipeline([
    ('cvec_a', CountVectorizer()),
    ('lr_a', LogisticRegression(solver= 'lbfgs'))
])

params_a = {
    'cvec_a__max_features': [2000, 3000, 4000, 5000],
    'cvec_a__min_df': [2, 3],
    'cvec_a__max_df': [.9, .95],
    'cvec_a__ngram_range': [(1,1), (1,4)],
    'cvec_a__stop_words':['english', None]
}

gs_a = GridSearchCV(pipe_a,params_a, cv=5, n_jobs=3)
    
gs_a.fit(X_train, y_train)
    
print(f'Gridsearch with Count Vectorizer for training data is {gs_a.score(X_train, y_train)}')
print()
print(f'Gridsearch with Count Vectorizer for test data is {gs_a.score(X_test, y_test)}')

Gridsearch with Count Vectorizer for training data is 0.993368700265252

Gridsearch with Count Vectorizer for test data is 0.8928571428571429


In [40]:
gs_a.best_params_

{'cvec_a__max_df': 0.9,
 'cvec_a__max_features': 2000,
 'cvec_a__min_df': 2,
 'cvec_a__ngram_range': (1, 1),
 'cvec_a__stop_words': None}

## LOGIST REGRESSION WITH TFIDF VECTORIZER

In [46]:
pipe_b = Pipeline([
    ('tfid_b', TfidfVectorizer()),
    ('lr_b', LogisticRegression(solver= 'lbfgs'))
])

params_b = {
    'tfid_b__max_features': [2000, 3000, 4000, 5000],
    'tfid_b__min_df': [2, 3],
    'tfid_b__max_df': [.9, .95],
    'tfid_b__ngram_range': [(1,1), (1,4)],
    'tfid_b__stop_words':['english', None]
}

gs_b = GridSearchCV(pipe_b,params_b, cv=5, n_jobs=3)
    
gs_b.fit(X_train, y_train)
    
print(f'Gridsearch with TFIDF Vectorizer for training data is {gs_b.score(X_train, y_train)}')
print()
print(f'Gridsearch with TFIDF Vectorizer for test data is {gs_b.score(X_test, y_test)}')

Gridsearch with TFIDF Vectorizer for training data is 0.8952254641909815

Gridsearch with TFIDF Vectorizer for test data is 0.8650793650793651


In [48]:
gs_b.best_params_

{'tfid_b__max_df': 0.9,
 'tfid_b__max_features': 2000,
 'tfid_b__min_df': 3,
 'tfid_b__ngram_range': (1, 1),
 'tfid_b__stop_words': 'english'}

## SVC WITH COUNT VECTORIZER

In [49]:
pipe_c = Pipeline([
    ('cvec_c', CountVectorizer()),
    ('svc', SVC(C=2,gamma='scale'))
])

params_c = {
    'cvec_c__max_features': [2000, 3000, 4000, 5000],
    'cvec_c__min_df': [2, 3],
    'cvec_c__max_df': [.9, .95],
    'cvec_c__ngram_range': [(1,1), (1,4)],
    'cvec_c__stop_words':['english', None]
}

gs_c = GridSearchCV(pipe_c,params_c, cv=5, n_jobs=3)
    
gs_c.fit(X_train, y_train)
    
print(f'Gridsearch with Count Vectorizer for training data is {gs_c.score(X_train, y_train)}')
print()
print(f'Gridsearch with Count Vectorizer for test data is {gs_c.score(X_test, y_test)}')

Gridsearch with Count Vectorizer for training data is 0.9960212201591512

Gridsearch with Count Vectorizer for test data is 0.9126984126984127


## SVC WITH TFIDF VECTORIZER

In [50]:
pipe_d = Pipeline([
    ('tfid_d', TfidfVectorizer()),
    ('svc', SVC(C=2,gamma='scale'))
])

params_d = {
    'tfid_d__max_features': [2000, 3000, 4000, 5000],
    'tfid_d__min_df': [2, 3],
    'tfid_d__max_df': [.9, .95],
    'tfid_d__ngram_range': [(1,1), (1,4)],
    'tfid_d__stop_words':['english', None]
}

gs_d = GridSearchCV(pipe_d,params_d, cv=5, n_jobs=3)
    
gs_d.fit(X_train, y_train)
    
print(f'Gridsearch with Count Vectorizer for training data is {gs_d.score(X_train, y_train)}')
print()
print(f'Gridsearch with Count Vectorizer for test data is {gs_d.score(X_test, y_test)}')

Gridsearch with Count Vectorizer for training data is 1.0

Gridsearch with Count Vectorizer for test data is 0.9087301587301587


## MULTINOMIAL NAIVE BAYES

In [27]:
pipe_e = Pipeline([
    ('cvec_e', CountVectorizer()),
    ('mnb', MultinomialNB())
])

params_e = {}

gs_e = GridSearchCV(pipe_e, params_e, cv=5, n_jobs=3)

gs_e.fit(X_train, y_train)

print(f'Gridsearch with Count Vectorizer for training data is {gs_e.score(X_train, y_train)}')
print(f'Gridsearch with Count Vectorizer for test data is {gs_e.score(X_train, y_train)}')

Gridsearch with Count Vectorizer for training data is 0.980106100795756
Gridsearch with Count Vectorizer for test data is 0.980106100795756


In [28]:
from sklearn.metrics import confusion_matrix

In [32]:
preds = gs_e.predict(X_test)

In [33]:
# Generate confusion matrix.
# Documentation here: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
# tn, fp  positive = asking for help
# fn, tp  negative = not asking for help
confusion_matrix(y_test, # True values.
                 preds)  # Predicted values.

array([[186,   9],
       [ 16,  41]])

## GAUSSIAN NAIVE BAYES

In [11]:
pipe_f = Pipeline([
    ('tfid_e', TfidfVectorizer()),
    ('mnb', MultinomialNB())
])

params_f = {}

gs_f = GridSearchCV(pipe_f, params_f, cv=5, n_jobs=3)

gs_f.fit(X_train, y_train)

print(f'Gridsearch with Tfidf Vectorizer for training data is {gs_f.score(X_train, y_train)}')
print(f'Gridsearch with Tfidf Vectorizer for training data is {gs_f.score(X_train, y_train)}')

Gridsearch with Tfidf Vectorizer for training data is 0.8355437665782494
Gridsearch with Tfidf Vectorizer for training data is 0.8355437665782494
